In [12]:
import pandas as pd
from datetime import date
import PyPDF2
from pdfminer.high_level import extract_text
from tqdm.notebook import tqdm

In [13]:
current_session = "session_15"

In [14]:
ss = pd.read_csv(f'sessions/{current_session}/{current_session}.csv',
                 parse_dates=['date'], infer_datetime_format=True)
ss.date = pd.to_datetime(ss.date).dt.date
sessions = ss.session.tolist()
session_date = dict(zip(ss.session,ss.date))

C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\208375682.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  ss = pd.read_csv(f'sessions/{current_session}/{current_session}.csv',
C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\208375682.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ss = pd.read_csv(f'sessions/{current_session}/{current_session}.csv',


In [15]:
ss.head()

,session,date
0,15-1-1-1,2022-12-19
1,15-1-1-2,2022-12-20
2,15-2-1-1,2023-02-13
3,15-2-1-2,2023-02-14
4,15-2-1-3,2023-02-15


In [16]:
mp = pd.read_csv(f"sessions/{current_session}/mp_{current_session}.csv")
mp.head()

,seat_code,state,seat,mp,current_party
0,P001,Perlis,Padang Besar,RUSHDAN BIN RUSMI,PAS
1,P002,Perlis,Kangar,ZAKRI BIN HASSAN,BERSATU
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS
3,P004,Kedah,Langkawi,MOHD SUHAIMI BIN ABDULLAH,BERSATU
4,P005,Kedah,Jerlun,ABDUL GHANI BIN AHMAD,PAS


In [17]:
mp = mp.dropna() # for situation that dont have enough MPs, such as sesions 14
mp_list = [name.lower().replace(" ", "") for name in mp["mp"].tolist()]
len(mp_list)

222

In [18]:
area_list = [area.lower().replace(" ", "") for area in mp["seat"].tolist()]
area_list[:5]

['padangbesar', 'kangar', 'arau', 'langkawi', 'jerlun']

In [19]:
name_decode = [name.lower().replace(" ","") for name in mp["mp"].tolist()]

In [20]:
mp["seat_decode"] = area_list
mp["name_decode"] = name_decode

In [21]:
mp.head(3)

,seat_code,state,seat,mp,current_party,seat_decode,name_decode
0,P001,Perlis,Padang Besar,RUSHDAN BIN RUSMI,PAS,padangbesar,rushdanbinrusmi
1,P002,Perlis,Kangar,ZAKRI BIN HASSAN,BERSATU,kangar,zakribinhassan
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS,arau,shahidanbinkassim


In [22]:
seat_code = [code for code in mp["seat_code"].tolist()]
seat_code[:5]

['P001', 'P002', 'P003', 'P004', 'P005']

### Strategy

- Iterate through all pdfs
- Search pages with **"Dewan riuh"**
- Find who spoke right before riuh moment
    - 2nd
    - 3rd


In [23]:
session_dict = dict()
for session in tqdm(sessions):
    pdf_active = PyPDF2.PdfReader(open(f'src_hansard/{current_session}/hansard_' + session + '.pdf', 'rb', ),strict=False)
    n_pages = len(pdf_active.pages)
    page_number = []
    who_list = []
    riuh_count = dict()

    for page in range(n_pages):
        page_active = ''.join(pdf_active.pages[page].extract_text().split()).lower()
        # search for pages with dewan riuh
        if "dewanriuh" in page_active:
            page_number.append(page)
            # append page before, if the page is still not in the list of page_number --> sometimes dewanriuh at top of the page
            b_page_num = page - 1
            if b_page_num not in page_number:
                page_number.append(b_page_num)

    res = extract_text(f'src_hansard/{current_session}/hansard_' + session + '.pdf',
                       page_numbers=page_number).replace("\n", "%")
    res = ''.join(res.split()).lower()
    res = res.split("dewanriuh")

    for chunk in res[:-1]: # only find who involves before dewan riuh
        temps = []
        last_idx = chunk.rfind(":")
        first_idx = chunk[:last_idx].rfind("%")
        if last_idx != -1:
            temp = chunk[first_idx + 1 : last_idx]
            temps.append(temp)
            for i in range(5): # max possible combination including ydp
                last_idx = chunk[:first_idx].rfind(':') # start find the : sign before the (2nd occurance of) % sign
                if last_idx != -1:
                    first_idx = chunk[:last_idx].rfind('%')
                    temp = chunk[first_idx + 1 : last_idx]
                    temps.append(temp)

        # remove noise from the results list
        while "tuanyangdi-pertua" in temps: # remove ydp
            temps.remove("tuanyangdi-pertua")

        while "sepertiberikut" in temps: # remove seperti berikut
            temps.remove("sepertiberikut")

        while "seorangahli" in temps: # remove unknown
            temps.remove("seorangahli")

        temps = temps[:3]
        who_list += temps


    for item in who_list:
            if "tuanyangdi-pertua" in item or "azalina" in item: # count ydp [& azalina (bugs)!]
                if item in riuh_count.keys():
                    riuh_count[item] += 1
                elif item not in riuh_count.keys():
                    riuh_count[item] = 1

            elif "[" and "]" in item: # count person with area tag
                for area in area_list:
                    if area in item: # check if the area in the list
                        if area in riuh_count.keys():
                            riuh_count[area] += 1
                        elif area not in riuh_count.keys():
                            riuh_count[area] = 1
            else:
                for name in mp_list:
                    if name in item: # check if the name in the list [there are bugs here, some name doesnt match]
                        if name in riuh_count.keys():
                            riuh_count[name] += 1
                        elif name not in riuh_count.keys():
                            riuh_count[name] = 1

    session_dict[session_date[session]] = riuh_count

  0%|          | 0/33 [00:00<?, ?it/s]

In [24]:
mega_sum = 0
for i in session_dict.values():
    session_sum = (sum(i.values()))
    mega_sum += session_sum
    
print("Number of session:", len(session_dict.values()))

assert mega_sum % 3 == 0, "Problem on riuh distribution"
print("Total riuh:" ,mega_sum / 3)

Number of session: 33
Total riuh: 193.0


In [25]:
df = pd.DataFrame(session_dict.values(),
                  index=session_dict.keys())
df = df.fillna(0)
df = df.iloc[:,:].astype("int")
df["pengerang"] = df["dato'seriazalinaothmansaid"] + df["dato’seriazalinaothmansaid"] #bugs on azalina name
df = df.drop(["dato’seriazalinaothmansaid", "dato'seriazalinaothmansaid"], axis=1)
for i in df.columns:
    for area in area_list:
        if i == area:
            mask = mp["seat_decode"] == area
            get_seat_code = mp[mask].seat_code.values[0]
            # Rename the i column to the seat code
            df = df.rename(columns={i: get_seat_code})
    
    for name in name_decode:
        if i == name:
            mask = mp["name_decode"] == name
            get_seat_code = mp[mask].seat_code.values[0]
            # Rename the i column to the seat code
            df = df.rename(columns={i: get_seat_code})

df = df.transpose()
df = df.groupby(df.index).sum() # remove duplicate
df['total_riuh'] = df.sum(axis=1)
df = df.reset_index().rename(columns={"index": "seat_code"})

In [26]:
mp = mp.drop(["seat_decode", "name_decode"], axis=1)
mp_riuh = mp.merge(df, on="seat_code", how="outer")
mp_riuh = mp_riuh.fillna(0)
mp_riuh = mp_riuh.convert_dtypes()
mp_riuh.head(3)

,seat_code,state,seat,mp,current_party,2022-12-19,2022-12-20,2023-02-13,2023-02-14,2023-02-15,...,2023-03-21,2023-03-22,2023-03-23,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-04-03,2023-04-04,total_riuh
0,P001,Perlis,Padang Besar,RUSHDAN BIN RUSMI,PAS,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,P002,Perlis,Kangar,ZAKRI BIN HASSAN,BERSATU,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS,7,3,0,0,1,...,4,0,1,2,2,1,0,4,3,57


In [27]:
mp_riuh.sort_values(by="total_riuh", ascending=False)

,seat_code,state,seat,mp,current_party,2022-12-19,2022-12-20,2023-02-13,2023-02-14,2023-02-15,...,2023-03-21,2023-03-22,2023-03-23,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-04-03,2023-04-04,total_riuh
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS,7,3,0,0,1,...,4,0,1,2,2,1,0,4,3,57
10,P011,Kedah,Pendang,AWANG BIN HASHIM,PAS,6,0,0,0,0,...,0,0,0,3,0,1,0,11,0,45
49,P050,Pulau Pinang,Jelutong,SANISVARA NETHAJI RAYER A/L RAJAJI,DAP,3,0,0,1,7,...,0,0,0,4,0,1,0,7,0,32
100,P101,Selangor,Hulu Langat,MOHD SANY HAMZAN,AMANAH,1,0,0,0,3,...,1,0,0,1,0,0,0,0,0,26
69,P070,Perak,Kampar,CHONG ZHEMIN,DAP,0,0,0,0,0,...,0,0,0,0,0,0,17,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,P110,Selangor,Klang,GANABATIRAU A/L VERAMAN,DAP,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112,P113,Selangor,Sepang,RAJ MUNNI SABU @ AIMAN ATHIRAH AL JUNDI,AMANAH,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,P116,W.P. Kuala Lumpur,Wangsa Maju,ZAHIR HASSAN,PKR,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,P117,W.P. Kuala Lumpur,Segambut,HANNAH YEOH TSEOW SUAN,DAP,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
mp_riuh.to_csv(f'sessions/{current_session}/riuh_stats_{current_session}.csv',index=False)

## dewan ketawa

Find only the one who cause the dewan laugh

In [30]:
g_session_dict = dict()
for session in tqdm(sessions):
    pdf_active = PyPDF2.PdfReader(open(f'src_hansard/{current_session}/hansard_' + session + '.pdf', 'rb', ),strict=False)
    n_pages = len(pdf_active.pages)
    page_number = []
    who_list = []
    gelak_count = dict()

    for page in range(n_pages):
        page_active = ''.join(pdf_active.pages[page].extract_text().split()).lower()
        # search for pages with dewan ketawa
        if "dewanketawa" in page_active:
            page_number.append(page)
            # append page before, if the page is still not in the list of page_number --> sometimes dewanketawa at top of the page
            b_page_num = page - 1
            if b_page_num not in page_number:
                page_number.append(b_page_num)

    res = extract_text(f'src_hansard/{current_session}/hansard_' + session + '.pdf',
                       page_numbers=page_number).replace("\n", "%")
    res = ''.join(res.split()).lower()
    res = res.split("dewanketawa")

    for chunk in res[:-1]: # only find who cause dewan ketawa
        temps = []
        last_idx = chunk.rfind(":")
        first_idx = chunk[:last_idx].rfind("%")
        if last_idx != -1:
            temp = chunk[first_idx + 1 : last_idx]
            temps.append(temp)
            #for i in range(5): # max possible combination including ydp
            #    last_idx = chunk[:first_idx].rfind(':') # start find the : sign before the (2nd occurance of) % sign
            #    if last_idx != -1:
            #        first_idx = chunk[:last_idx].rfind('%')
            #        temp = chunk[first_idx + 1 : last_idx]
            #        temps.append(temp)

        # remove noise from the results list
        while "tuanyangdi-pertua" in temps: # remove ydp
            temps.remove("tuanyangdi-pertua")

        while "sepertiberikut" in temps: # remove seperti berikut
            temps.remove("sepertiberikut")

        while "seorangahli" in temps: # remove unknown
            temps.remove("seorangahli")

        temps = temps[:1]
        who_list += temps


    for item in who_list:
            if "tuanyangdi-pertua" in item or "azalina" in item: # count ydp [& azalina (bugs)!]
                if item in gelak_count.keys():
                    gelak_count[item] += 1
                elif item not in gelak_count.keys():
                    gelak_count[item] = 1

            elif "[" and "]" in item: # count person with area tag
                for area in area_list:
                    if area in item: # check if the area in the list
                        if area in gelak_count.keys():
                            gelak_count[area] += 1
                        elif area not in gelak_count.keys():
                            gelak_count[area] = 1
            else:
                for name in mp_list:
                    if name in item: # check if the name in the list [there are bugs here, some name doesnt match]
                        if name in gelak_count.keys():
                            gelak_count[name] += 1
                        elif name not in gelak_count.keys():
                            gelak_count[name] = 1

    g_session_dict[session_date[session]] = gelak_count

  0%|          | 0/33 [00:00<?, ?it/s]

In [44]:
mega_sum = 0
for i in g_session_dict.values():
    mega_sum += (sum(i.values()))

print("Number of session:", len(g_session_dict.keys()))
print("Total gelak:" , mega_sum)

Number of session: 33
Total gelak: 55


In [62]:
mp["seat_decode"] = area_list
mp["name_decode"] = name_decode

In [63]:
df = pd.DataFrame(g_session_dict.values(),
                  index=g_session_dict.keys())
df = df.fillna(0)
df = df.iloc[:,:].astype("int")
df["pengerang"] = df["dato'seriazalinaothmansaid"] + df["dato’seriazalinaothmansaid"] #bugs on azalina name
df = df.drop(["dato’seriazalinaothmansaid", "dato'seriazalinaothmansaid"], axis=1)
for i in df.columns:
    for area in area_list:
        if i == area:
            mask = mp["seat_decode"] == area
            get_seat_code = mp[mask].seat_code.values[0]
            # Rename the i column to the seat code
            df = df.rename(columns={i: get_seat_code})
    
    for name in name_decode:
        if i == name:
            mask = mp["name_decode"] == name
            get_seat_code = mp[mask].seat_code.values[0]
            # Rename the i column to the seat code
            df = df.rename(columns={i: get_seat_code})

df = df.transpose()
df = df.groupby(df.index).sum() # remove duplicate
df['total_gelak'] = df.sum(axis=1)
df = df.reset_index().rename(columns={"index": "seat_code"})
mp = mp.drop(["seat_decode", "name_decode"], axis=1)

In [64]:
df

,seat_code,2022-12-19,2022-12-20,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-20,2023-02-21,2023-02-22,...,2023-03-21,2023-03-22,2023-03-23,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-04-03,2023-04-04,total_gelak
0,P003,0,1,0,0,0,0,0,1,0,...,3,0,1,3,0,0,0,0,5,23
1,P004,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,P013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,P022,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,P028,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,P034,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,3
6,P050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,P056,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
8,P057,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9,P063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [65]:
mp_gelak = mp.merge(df, on="seat_code", how="outer")
mp_gelak = mp_gelak.fillna(0)
mp_gelak = mp_gelak.convert_dtypes()
mp_gelak.head(3)

,seat_code,state,seat,mp,current_party,2022-12-19,2022-12-20,2023-02-13,2023-02-14,2023-02-15,...,2023-03-21,2023-03-22,2023-03-23,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-04-03,2023-04-04,total_gelak
0,P001,Perlis,Padang Besar,RUSHDAN BIN RUSMI,PAS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,P002,Perlis,Kangar,ZAKRI BIN HASSAN,BERSATU,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS,0,1,0,0,0,...,3,0,1,3,0,0,0,0,5,23


In [68]:
mp_gelak.sort_values(by="total_gelak", ascending=False).head(10)

,seat_code,state,seat,mp,current_party,2022-12-19,2022-12-20,2023-02-13,2023-02-14,2023-02-15,...,2023-03-21,2023-03-22,2023-03-23,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-04-03,2023-04-04,total_gelak
2,P003,Perlis,Arau,SHAHIDAN BIN KASSIM,PAS,0,1,0,0,0,...,3,0,1,3,0,0,0,0,5,23
156,P157,Johor,Pengerang,AZALINA BINTI OTHMAN,UMNO,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,P004,Kedah,Langkawi,MOHD SUHAIMI BIN ABDULLAH,BERSATU,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
100,P101,Selangor,Hulu Langat,MOHD SANY HAMZAN,AMANAH,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3
33,P034,Terengganu,Setiu,SHAHARIZUKIRNAIN BIN ABD KADIR,PAS,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,3
55,P056,Perak,Larut,HAMZAH BIN ZAINUDIN,BERSATU,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
62,P063,Perak,Tambun,ANWAR BIN IBRAHIM,PKR,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
186,P187,Sabah,Kinabatangan,MOKTAR BIN RADIN,UMNO,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
148,P149,Johor,Sri Gading,AMINOLHUDA BIN HASSAN,AMANAH,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
95,P096,Selangor,Kuala Selangor,DZULKEFLY BIN AHMAD,AMANAH,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [67]:
mp_gelak.to_csv(f'sessions/{current_session}/gelak_stats_{current_session}.csv',index=False)